In [106]:
import sqlite3

In [107]:
DB_PATH = "databases/carsharing.db"

In [108]:
connection = sqlite3.connect(DB_PATH)

##### Создание баз данных

In [109]:
cursor = connection.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Car(
        id INTEGER PRIMARY KEY,
        brand TEXT,
        license_plates TEXT,
        fuel INTEGER,
        location TEXT
    );
''')

cars = [
        ('Mazda CX-9', 'Е103ВЛ', 512, 'Ленинский пр-т, 109'),
        ('Mazda 6', 'П706ВД', 157, 'ул. Веерная, 30'),
        ('Nissan Qashqai', 'Р800НГ', 301, 'ул. Маршала Василевского, 7к2'),
        ('Scoda Octavia', 'Ш871АУ', 243, 'ул. Ладожская, 2/37'),
        ('Mercedes-Benz E200', 'У901ВФ', 289, 'ул. Башиловская, 19')
]

cursor.executemany(
    """INSERT INTO Car(brand, license_plates, fuel, location)
       VALUES (?, ?, ?, ?);""",
    cars
)

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Driver(
        id INTEGER PRIMARY KEY,
        name TEXT,
        age INTEGER,
        experience REAL
    );
''')

drivers = [
        ('Е. Виноградов', 27, 1.2),
        ('Н. Пузанков', 31, 10.5),
        ('К. Ковалёв', 21, 1.0),
        ('И. Заречный', 35, 5.7)
]

cursor.executemany(
    """INSERT INTO Driver(name, age, experience)
       VALUES (?, ?, ?);""",
    drivers
)

cursor.execute('''
        CREATE TABLE IF NOT EXISTS Renting(
        id INTEGER PRIMARY KEY,
        driver_id INTEGER,
        car_id INTEGER,
        cost INTEGER,
        date TEXT,
        start_time TEXT,
        FOREIGN KEY (driver_id) REFERENCES Driver (id),
        FOREIGN KEY (car_id) REFERENCES Car (id)
    );
''')


rentings = [
    (1, 1, 618, '2023-03-01', '08:15:57'),
    (2, 4, 507, '2023-03-18', '10:27:41'),
    (1, 3, 640, '2023-03-18', '21:19:10'),
    (3, 3, 328, '2023-04-10', '01:15:34')
]

cursor.executemany(
    """INSERT INTO Renting(driver_id, car_id, cost, date, start_time)
       VALUES (?, ?, ?, ?, ?);""",
    rentings
)
connection.commit()

##### Вывод всех строк баз данных

In [110]:
def output_all(connection, table_name: str) -> None:
    cursor = connection.cursor()

    for row in cursor.execute(f"SELECT * FROM {table_name};"):
        print(row)

In [111]:
output_all(connection, 'Car')

(1, 'Mazda CX-9', 'Е103ВЛ', 512, 'Ленинский пр-т, 109')
(2, 'Mazda 6', 'П706ВД', 157, 'ул. Веерная, 30')
(3, 'Nissan Qashqai', 'Р800НГ', 301, 'ул. Маршала Василевского, 7к2')
(4, 'Scoda Octavia', 'Ш871АУ', 243, 'ул. Ладожская, 2/37')
(5, 'Mercedes-Benz E200', 'У901ВФ', 289, 'ул. Башиловская, 19')


In [112]:
output_all(connection, 'Driver')

(1, 'Е. Виноградов', 27, 1.2)
(2, 'Н. Пузанков', 31, 10.5)
(3, 'К. Ковалёв', 21, 1.0)
(4, 'И. Заречный', 35, 5.7)


In [113]:
output_all(connection, 'Renting')

(1, 1, 1, 618, '2023-03-01', '08:15:57')
(2, 2, 4, 507, '2023-03-18', '10:27:41')
(3, 1, 3, 640, '2023-03-18', '21:19:10')
(4, 3, 3, 328, '2023-04-10', '01:15:34')


##### Вывод марок машин

In [114]:
def print_cars(connection):
    cursor = connection.cursor()
    for row in cursor.execute("SELECT brand FROM Car;"):
        print(row[0])

In [115]:
print_cars(connection)

Mazda CX-9
Mazda 6
Nissan Qashqai
Scoda Octavia
Mercedes-Benz E200


##### Вывод имён пользователей

In [116]:
def print_drivers(connection):
    cursor = connection.cursor()
    for row in cursor.execute("SELECT name FROM Driver;"):
        print(row[0])

In [117]:
print_drivers(connection)

Е. Виноградов
Н. Пузанков
К. Ковалёв
И. Заречный


##### Возможность добавления новой машины

In [118]:
def add_car(connection, car: tuple):
    cursor = connection.cursor()
    cursor.executemany(
    """INSERT INTO Car(brand, fuel, location)
       VALUES (?, ?, ?);""",
    car
    )
    connection.commit()

In [119]:
car = [('Hyundai Creta', 333, 'пр-т 60-летия Октября, 20'),
       ('Renault Duster', 414, 'ул. Дмитрия Ульянова, 13к1')]

In [120]:
add_car(connection, car)
output_all(connection, 'Car')

(1, 'Mazda CX-9', 'Е103ВЛ', 512, 'Ленинский пр-т, 109')
(2, 'Mazda 6', 'П706ВД', 157, 'ул. Веерная, 30')
(3, 'Nissan Qashqai', 'Р800НГ', 301, 'ул. Маршала Василевского, 7к2')
(4, 'Scoda Octavia', 'Ш871АУ', 243, 'ул. Ладожская, 2/37')
(5, 'Mercedes-Benz E200', 'У901ВФ', 289, 'ул. Башиловская, 19')
(6, 'Hyundai Creta', None, 333, 'пр-т 60-летия Октября, 20')
(7, 'Renault Duster', None, 414, 'ул. Дмитрия Ульянова, 13к1')


##### Возможность добавления пользователя

In [121]:
def add_driver(connection, driver:tuple):
    cursor = connection.cursor()
    cursor.executemany(
    """INSERT INTO Driver(name, age, experience)
       VALUES (?, ?, ?);""",
    driver
    )
    connection.commit()

In [122]:
driver = [('С. Иванов', 45, 14.9)]

In [123]:
add_driver(connection, driver)
output_all(connection, 'Driver')

(1, 'Е. Виноградов', 27, 1.2)
(2, 'Н. Пузанков', 31, 10.5)
(3, 'К. Ковалёв', 21, 1.0)
(4, 'И. Заречный', 35, 5.7)
(5, 'С. Иванов', 45, 14.9)


##### Возможность добавления аренды машины

In [124]:
def give_avto(connection, driver, car: tuple, cost, date, start_time, new_loc):
    cursor = connection.cursor()
    
    cursor.execute(f"SELECT * FROM Driver WHERE name = '{driver}'")
    driver_id = cursor.fetchone()[0]
    cursor.execute(f"SELECT * FROM Car WHERE brand = '{car[0]}' AND license_plates = '{car[1]}'")
    car_id = cursor.fetchone()[0]
    
# Наверное можно так сделать, но это в случае, если в базу данных сразу вносится информация о заключении аренды
#     from datetime import datetime
#     date = datetime.today().strftime('%Y-%m-%d')
    
    value = [(driver_id, car_id, cost, date, start_time)]
    cursor.executemany(
    """INSERT INTO Renting(driver_id, car_id, cost, date, start_time)
       VALUES (?, ?, ?, ?, ?);""",
    value
    )
    
    
    cursor.execute(
    f"UPDATE Car SET location = '{new_loc}' WHERE brand = '{car[0]}' AND license_plates = '{car[1]}'",
    )
    
    
    connection.commit()

In [125]:
give_avto(connection, 'И. Заречный', ('Mercedes-Benz E200', 'У901ВФ'), 580, '2023-04-14', '15:04:45', 'ул. Вавилова, 82')

In [126]:
output_all(connection, 'Renting')

(1, 1, 1, 618, '2023-03-01', '08:15:57')
(2, 2, 4, 507, '2023-03-18', '10:27:41')
(3, 1, 3, 640, '2023-03-18', '21:19:10')
(4, 3, 3, 328, '2023-04-10', '01:15:34')
(5, 4, 5, 580, '2023-04-14', '15:04:45')


In [127]:
output_all(connection, 'Car')

(1, 'Mazda CX-9', 'Е103ВЛ', 512, 'Ленинский пр-т, 109')
(2, 'Mazda 6', 'П706ВД', 157, 'ул. Веерная, 30')
(3, 'Nissan Qashqai', 'Р800НГ', 301, 'ул. Маршала Василевского, 7к2')
(4, 'Scoda Octavia', 'Ш871АУ', 243, 'ул. Ладожская, 2/37')
(5, 'Mercedes-Benz E200', 'У901ВФ', 289, 'ул. Вавилова, 82')
(6, 'Hyundai Creta', None, 333, 'пр-т 60-летия Октября, 20')
(7, 'Renault Duster', None, 414, 'ул. Дмитрия Ульянова, 13к1')


##### Удаление пользователя (например, лишили водительских прав)

In [88]:
def del_user(connection, driver):
    cursor.execute(f"SELECT * FROM Driver WHERE name = '{driver}'")
    driver_id = cursor.fetchone()[0]
    
    cursor.execute(f"DELETE FROM Renting WHERE driver_id = '{driver_id}'")
    cursor.execute(f"DELETE FROM Driver WHERE name = '{driver}'")
    
    connection.commit()

In [89]:
del_user(connection, 'Н. Пузанков')

In [90]:
output_all(connection, 'Driver')

(1, 'Е. Виноградов', 27, 1.2)
(3, 'К. Ковалёв', 21, 1.0)
(4, 'И. Заречный', 35, 5.7)
(5, 'С. Иванов', 45, 14.9)


In [91]:
output_all(connection, 'Renting')

(1, 1, 1, 618, '2023-03-01', '08:15:57')
(3, 1, 3, 640, '2023-03-18', '21:19:10')
(4, 3, 3, 328, '2023-04-10', '01:15:34')
(5, 4, 5, 580, '2023-04-14', '15:04:45')


##### Удаление автомобиля (возможно ремонт)

In [98]:
def del_avto(connection, car):
    cursor.execute(f"SELECT * FROM Car WHERE brand = '{car}'")
    car_id = cursor.fetchone()[0]
    
    cursor.execute(f"DELETE FROM Renting WHERE car_id = '{car_id}'")
    cursor.execute(f"DELETE FROM Car WHERE brand = '{car}'")
    
    connection.commit()

In [99]:
del_avto(connection, 'Mazda CX-9')

In [100]:
output_all(connection, 'Car')

(2, 'Mazda 6', 'П706ВД', 157, 'ул. Веерная, 30')
(3, 'Nissan Qashqai', 'Р800НГ', 301, 'ул. Маршала Василевского, 7к2')
(4, 'Scoda Octavia', 'Ш871АУ', 243, 'ул. Ладожская, 2/37')
(5, 'Mercedes-Benz E200', 'У901ВФ', 289, 'ул. Вавилова, 82')
(6, 'Hyundai Creta', None, 333, 'пр-т 60-летия Октября, 20')
(7, 'Renault Duster', None, 414, 'ул. Дмитрия Ульянова, 13к1')


In [101]:
output_all(connection, 'Renting')

(3, 1, 3, 640, '2023-03-18', '21:19:10')
(4, 3, 3, 328, '2023-04-10', '01:15:34')
(5, 4, 5, 580, '2023-04-14', '15:04:45')


##### Создал заново таблицу и добавил всё выше, при этом ничего не удаляя

##### Подсчёт общего числа элементов в таблице

In [152]:
def quantity(connection, table):
    cursor.execute(f"SELECT * FROM '{table}'")
    print(len(cursor.fetchall()))

##### Общее количество автомобилей в парке

In [153]:
quantity(connection, 'Car')

7


##### Общее количество пользователей

In [154]:
quantity(connection, 'Driver')

5


##### Пользователь, который чаще всего пользуется услугами аренды автомобилей

In [186]:
cursor.execute("""  SELECT
                        driver_id,
                    COUNT(driver_id) AS `value_occurrence` 
                    FROM
                        'Renting'
                    GROUP BY 
                        driver_id

                    ORDER BY 
                          `value_occurrence` DESC""")
cursor.execute(f"SELECT * FROM 'Driver' WHERE ID = '{cursor.fetchall()[0][0]}'")
cursor.fetchone()[1]

'Е. Виноградов'

##### Автомобиль, который чаще всего используется

In [187]:
cursor.execute("""  SELECT
                        car_id,
                    COUNT(car_id) AS `value_occurrence` 
                    FROM
                        'Renting'
                    GROUP BY 
                        car_id
                    ORDER BY 
                          `value_occurrence` DESC""")
cursor.execute(f"SELECT * FROM 'Car' WHERE ID = '{cursor.fetchall()[0][0]}'")
cursor.fetchone()[1]

'Nissan Qashqai'

##### Автомобиль, который дольше всего используется

In [191]:
cursor.execute("SELECT car_id, SUM(cost) FROM 'Renting' GROUP BY car_id ORDER BY cost")
cursor.execute(f"SELECT * FROM 'Car' WHERE ID = '{cursor.fetchall()[-1][0]}'")
cursor.fetchone()[1]

'Nissan Qashqai'